In [1]:
import numpy as np
import cv2
import os
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split
from tensorflow.python.client import device_lib
from tensorflow.keras.preprocessing.image import ImageDataGenerator
fileRoot = "D:\\ICLR-CropDisease\\dataset\\"
zipsize =200


In [2]:
def blur(image):
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]], np.float32)
    return cv2.filter2D(image, -1, kernel=kernel)
def preprocess(res):
    #zipsize = 256
    res = cv2.resize(res, dsize=(zipsize, zipsize))
    res = blur(res)
    #res = res [64:192,64:192]
    return res

def loadData():
    datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest')
    data_img = []
    data_label = []
    for file in os.listdir(fileRoot + "train\\healthy_wheat\\"):
        img = cv2.imread(fileRoot + "train\\healthy_wheat\\" + file)
        res = preprocess(img)
        data_img.append(res)
        data_label.append(0)
        data_img.append(cv2.flip(res, 1))
        data_label.append(0)
        data_img.append(cv2.flip(res, 0))
        data_label.append(0)
        data_img.append(cv2.flip(res, -1))
        data_label.append(0)
        data_img.append(datagen.random_transform(res))
        data_label.append(0)
        data_img.append(datagen.random_transform(res))
        data_label.append(0)

    for file in os.listdir(fileRoot + "train\\leaf_rust\\"):
        img = cv2.imread(fileRoot + "train\\leaf_rust\\" + file)
        if img is None:
            print(file)
            continue
        res = preprocess(img)
        data_label.append(1)
        data_img.append(res)
        data_img.append(cv2.flip(res, 1))
        data_label.append(1)
        data_img.append(cv2.flip(res, 0))
        data_label.append(1)
        data_img.append(cv2.flip(res, -1))
        data_label.append(1)
        data_img.append(datagen.random_transform(res))
        data_label.append(1)
        data_img.append(datagen.random_transform(res))
        data_label.append(1)

    for file in os.listdir(fileRoot + "train\\stem_rust\\"):
        img = cv2.imread(fileRoot + "train\\stem_rust\\" + file)
        res = preprocess(img)
        data_img.append(res)
        data_label.append(2)
        data_img.append(cv2.flip(res, 1))
        data_label.append(2)
        data_img.append(cv2.flip(res, 0))
        data_label.append(2)
        data_img.append(cv2.flip(res, -1))
        data_label.append(2)
        data_img.append(datagen.random_transform(res))
        data_label.append(2)
        data_img.append(datagen.random_transform(res))
        data_label.append(2)
        
    for i in range(len(data_img)):
        data_img[i] = data_img[i] / 255
    data_img = np.array(data_img)
    data_label = np.array(data_label)
    return data_img, data_label

In [3]:
def trainTestSplit(data_img,data_label):

    X_train, X_test, y_train, y_test = train_test_split(data_img,data_label,test_size = 0.3)
    return X_train, X_test, y_train, y_test

In [4]:
def transferModel(X_train, X_test, y_train, y_test):
    with tf.device('/gpu:0'):
        
        base_model = tf.keras.applications.VGG19(input_shape=(zipsize, zipsize, 3),
                                               include_top=False,
                                               weights='imagenet')
        for layer in base_model.layers[:20]:
            layer.trainable = False
        
        global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
        dense_layer = tf.keras.layers.Dense(128)
        drop_layer = tf.keras.layers.Dropout(0.3)
        prediction_layer = tf.keras.layers.Dense(3, activation='softmax')


        model = tf.keras.Sequential([base_model,global_average_layer,dense_layer,drop_layer,prediction_layer])
        print(model.summary())
        model.compile(optimizer='Adamax',
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                      metrics=['accuracy'])
        # Train
        history = model.fit(X_train, y_train,batch_size = 16, epochs=50,use_multiprocessing = True)

        # test
        test_loss, test_acc = model.evaluate(X_test,y_test, batch_size = 16, verbose=2)
        print(test_acc)
        return model

In [5]:
def trainModel(X_train, X_test, y_train, y_test):
   
    with tf.device('/gpu:0'):
        
        model = models.Sequential()
        model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(zipsize, zipsize, 3)))
        model.add(layers.Conv2D(32, (3, 3), activation='relu'))
        model.add(layers.Conv2D(32, (3, 3), activation='relu'))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Conv2D(32, (3, 3), activation='relu'))
        model.add(layers.Conv2D(32, (3, 3), activation='relu'))
        model.add(layers.Conv2D(32, (3, 3), activation='relu'))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Conv2D(64, (3, 3), activation='relu'))
        model.add(layers.Conv2D(64, (3, 3), activation='relu'))
        model.add(layers.Conv2D(64, (3, 3), activation='relu'))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Conv2D(64, (3, 3), activation='relu'))
        model.add(layers.Conv2D(64, (3, 3), activation='relu'))
        model.add(layers.Conv2D(64, (3, 3), activation='relu'))
        model.add(layers.Flatten())
        model.add(layers.Dense(128, activation='relu'))
        model.add(tf.keras.layers.Dropout(0.5))
        model.add(layers.Dense(3,activation='softmax'))

        model.compile(optimizer='Adamax',
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                      metrics=['accuracy'])

        # Train
        history = model.fit(X_train, y_train,batch_size = 16, epochs=50,use_multiprocessing = True)

        # test
        test_loss, test_acc = model.evaluate(X_test,y_test, batch_size = 16, verbose=2)
        print(test_acc)
        return model

In [6]:
data_img,data_lable = loadData()
print("DATA READY")

7U06EV.gif
DATA READY


In [7]:
X_train, X_test, y_train, y_test = trainTestSplit(data_img,data_lable)

model = transferModel(X_train, X_test, y_train, y_test)
#data_label = np.array(data_lable)
#model = trainModel(data_img,[],data_label,[])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 6, 6, 512)         20024384  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               65664     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 387       
Total params: 20,090,435
Trainable params: 2,425,859
Non-trainable params: 17,664,576
_________________________________________________________________
None
Train on 3675 samples
Epoch 1/50
  16/3675 [..............................] - ETA: 13:58

UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential/vgg19/block1_conv1/Conv2D (defined at C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\framework\ops.py:1751) ]] [Op:__inference_distributed_function_1910]

Function call stack:
distributed_function


In [ ]:
def loadTest():
    test = []
    name = []
    for file in os.listdir(fileRoot + "test\\"):
        img = cv2.imread(fileRoot + "test\\" + file)
        res = preprocess(img)
        test.append(res)
        name.append(file)
    for i in range(len(test)):
        test[i] = test[i] / 255
    test = np.array(test)
    result = model.predict_proba(test)
    return name, result
name,result = loadTest()

In [ ]:
import pandas as pd


output = []
for i in range(len(result)):
    output.append(np.append(result[i],name[i][0:6]).tolist())
my_df = pd.DataFrame(output)
# change order of columns of csv file so the name is in the first column
#print (my_df)
my_df = my_df[[3, 0, 1, 2]]
my_df.columns = ['ID', 'leaf_rust', 'stem_rust', 'healthy_wheat']
my_df.to_csv('submission.csv', index=False)